this is code


In [ ]:
from matplotlib import pyplot as plt
from PIL import Image
import os
import numpy as np
import glob
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)


: 

In [ ]:
class TheDataset(Dataset):
    def __init__(self, labels_dir, samples_dir, transform:bool = None, label_transform:bool = None):
        self.samples_dir = samples_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.label_transform = label_transform

        self.rgb_files = sorted(glob.glob(os.path.join(samples_dir, '*')))
        self.ndvi_files = sorted(glob.glob(os.path.join(labels_dir, '*')))

    def __len__(self):
        return len(self.rgb_files)

    def __getitem__(self, idx):
        rgb_path = self.rgb_files[idx]
        image = Image.open(rgb_path).convert("RGB")
    
        ndvi_path = self.ndvi_files[idx]
        ndvi = Image.open(ndvi_path)

        if self.transform:
            image = self.transform(image)
        if self.label_transform:
            ndvi = self.label_transform(ndvi)  
        else:
            ndvi = transforms.ToTensor()(ndvi)

        return image, ndvi

    def classes(self):
        return self.data.classes

In [ ]:
transform = transforms.Compose({
    transforms.Resize((128,128)),
    transforms.ToTensor()
})
dataset = TheDataset(
    labels_dir='labels', samples_dir='samples',
)

In [ ]:
sample, label = dataset[1]
label

In [ ]:
len(dataset)

# Whoops

In [ ]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
class TheModel(nn.Module):
    def __init__(self, output_size=1):
        super(TheModel, self).__init__()
        self.base_model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=0)

        with torch.no_grad():
            dummy_input = torch.randn(1, 3, 128, 128)
            features = self.base_model(dummy_input)
            feature_dim = features.shape[1]

        self.regressor = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(feature_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, output_size)
        )

    def forward(self, x):
        return self.regressor(self.base_model(x))

In [ ]:
model = TheModel()
print(str(model)[:500])

In [ ]:
for images, labels in dataloader:
    outputs = model(images)
    break  
model(images).shape

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), learning_rate=0.001)
